In [ ]:
qlogin -q short.qg -l gpu=1[affinity=true],gputype=rtx8000

In [ ]:
#!/bin/bash

cd /well/win/users/hsv459/agemapper

module purge
module load Python/3.7.4-GCCcore-8.3.0
module load CUDA/10.1.243-GCC-8.3.0
module load cudnn/7.6.5.32-10.1
# module add fsl/5.0.11

# VISIBLE_CUDA_DEVICES=0,1

# determine Ivybridge or Skylake compatibility on this node
CPU_ARCHITECTURE=$(/apps/misc/utils/bin/get-cpu-software-architecture.py)
# CPU_ARCHITECTURE=general

# Error handling
if [[ ! $? == 0 ]]; then
  echo "Fatal error: Please send the following information to the BMRC team: Could not determine CPU software architecture on $(hostname)"
  exit 1
fi

# Activate the ivybridge or skylake version of your python virtual environment
source /well/win/users/hsv459/python/functionmapper-${CPU_ARCHITECTURE}/bin/activate
# source /well/win/users/hsv459/python/functionmapper-skylake/bin/activate

# continue to use your python venv as normal
   
ipython


In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from solver import Solver
from AgeMapper import AgeMapper
from utils.data_utils import get_datasets, get_datasets_dynamically
from utils.settings import Settings
import utils.data_evaluation as evaluations
from utils.misc import my_KLDivLoss

import os
import numpy as np
import torch
import torch.nn as nn
import logging
import utils.data_utils as data_utils
from utils.misc import create_folder
import pandas as pd
from AgeMapper import AgeMapper
from collections import OrderedDict

In [ ]:
file_name = 'AM0-100'

settings_file_name = file_name + '.ini'
evaluation_settings_file_name = file_name + '_eval.ini'

settings = Settings(settings_file_name)
settings_evaluation = Settings(evaluation_settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']
mapping_evaluation_parameters = settings_evaluation['MAPPING']

experiment_name = mapping_evaluation_parameters['experiment_name']

trained_model_path = "saved_models/" + experiment_name + ".pth.tar"

# prediction_output_path = experiment_name + "_predictions"
prediction_output_path = experiment_name + "_distribution_predictions"

data_directory = mapping_evaluation_parameters['data_directory']

dataset_sex = mapping_evaluation_parameters['dataset_sex']
dataset_size = mapping_evaluation_parameters['dataset_size']

prediction_output_statistics_name = mapping_evaluation_parameters['prediction_output_statistics_name']

if mapping_evaluation_parameters['dataset_type'] == 'validation':
    data_list = mapping_evaluation_parameters['male_validation']
    test_ages = mapping_evaluation_parameters['male_validation_age']
    test_age_labels = mapping_evaluation_parameters['male_validation_age_labels']
    prediction_output_statistics_name += '_validation.csv'
else:
    data_list = mapping_evaluation_parameters['male_test']
    test_ages = mapping_evaluation_parameters['male_test_age']
    test_age_labels = mapping_evaluation_parameters['male_test_age_labels']
    prediction_output_statistics_name += '_test.pkl'

if dataset_sex == 'female':
    data_list = "fe" + data_list
    test_ages = "fe" + test_ages
    test_age_labels = "fe" + test_age_labels

if dataset_size == 'small':
    data_list += "_small.txt"
    test_ages += "_small.npy"
    test_age_labels += "_small.npy"
elif dataset_size == 'tiny':
    data_list += "_tiny.txt"
    test_ages += "_tiny.npy"
    test_age_labels += "_tiny.npy"
else:
    data_list += ".txt"
    test_ages += ".npy"
    test_age_labels += ".npy"

data_list = 'datasets/' + data_list
test_ages = 'datasets/' + test_ages
test_age_labels = 'datasets/' + test_age_labels

modality_flag = mapping_evaluation_parameters['modality_flag']
control = mapping_evaluation_parameters['control']
device = mapping_evaluation_parameters['device']
exit_on_error = mapping_evaluation_parameters['exit_on_error']

bin_centers_path = mapping_evaluation_parameters['bin_centers']

In [ ]:
# def evaluate_data(trained_model_path,
#                      data_directory,
#                      data_list,
#                      prediction_output_path,
#                      prediction_output_statistics_name,
#                      modality_flag,
#                      control,
#                      device,
#                      test_ages,
#                      test_age_labels,
#                      bin_centers_path,
#                      dataset_sex,
#                      network_parameters,
#                      exit_on_error=False):


def _generate_age(file_path,
                model,
                device,
                cuda_available,
                modality_flag,
                ):

    volume = data_utils.load_and_preprocess_evaluation(file_path, modality_flag)

    if len(volume.shape) != 5:
        volume = volume[np.newaxis, np.newaxis, :, :, :]

    volume = torch.tensor(volume).type(torch.FloatTensor)

    if cuda_available and (type(device) == int):
        volume = volume.cuda(device)

    output = model(volume)
    output = (output.cpu().numpy()).astype('float32')
    output = np.squeeze(output)

    return output


def _load_target(volume_index, test_ages, test_age_labels):

    target_age = np.load(test_ages)[volume_index]

    return target_age

# evaluate_data(trained_model_path,
#                  data_directory,
#                  data_list,
#                  prediction_output_path,
#                  prediction_output_statistics_name,
#                  modality_flag,
#                  control,
#                  device,
#                  test_ages,
#                  test_age_labels,
#                  bin_centers_path,
#                  dataset_sex,
#                  network_parameters,
#                  exit_on_error
#                 )

with open(data_list) as data_list_file:
    volumes_to_be_used = data_list_file.read().splitlines()

model = AgeMapper(output_dimension = network_parameters['output_dimension'])
age_prediction_loss_flag = False

trained_model = torch.load(trained_model_path,map_location=torch.device(device))

if hasattr(trained_model, 'state_dict'):
    model_sd = trained_model.state_dict()
    if torch.cuda.device_count()>1 or device=='cpu':
        correct_state_dict = {}
        for key in model_sd.keys():
            if key.startswith('module.'):
                new_key = key.replace('module.', "")
                correct_state_dict[new_key] = model_sd[key]
            else:
                correct_state_dict[key] = model_sd[key]
        correct_state_dict = OrderedDict(correct_state_dict)
        del model_sd
    model.load_state_dict(correct_state_dict)
else:
    model.load_state_dict(trained_model)

del trained_model

if torch.cuda.is_available() == True and device!='cpu':
    model.cuda(device)
    cuda_available = torch.cuda.is_available()
else:
    cuda_available = 'False'

model.eval()

# Create the prediction path folder if this is not available

create_folder(prediction_output_path)

# Initiate the evaluation

if modality_flag == 'T1_nonlinear':
    mapping_data_file = "T1/T1_brain_to_MNI.nii.gz"
elif modality_flag == 'T1_linear':
    mapping_data_file = "T1/T1_brain_to_MNI_linear.nii.gz"
elif modality_flag == 'T2_nonlinear':
    mapping_data_file = "T2_FLAIR/T2_FLAIR_brain_to_MNI.nii.gz"
elif modality_flag == 'tracts':
    mapping_data_file = "dMRI/autoptx_preproc/tractsNormSummed.nii.gz"


file_paths, volumes_to_be_used = data_utils.load_file_paths(data_list, data_directory, mapping_data_file)

if control == 'mean':
    prediction_output_statistics_name = "output_statistics_mean_target.csv"
elif control == 'null':
    prediction_output_statistics_name = "output_statistics_null_target.csv"

output_statistics = {}
output_statistics_path = os.path.join(prediction_output_path, prediction_output_statistics_name)

bin_centers = np.load(bin_centers_path)

with torch.no_grad():

    for volume_index, file_path in enumerate(file_paths):
        try:
            print("\r Mapping Volume {:.3f}%: {}/{} test subjects".format((volume_index+1)/len(file_paths) * 100.0, volume_index+1, len(file_paths)), end='')

            # Generate volume & header

            subject = volumes_to_be_used[volume_index]

            output_label = _generate_age(file_path,
                                    model,
                                    device,
                                    cuda_available,
                                    modality_flag,
                                    )

#             print(output_label)

            if age_prediction_loss_flag == False:
                    target_age = _load_target(
                        volume_index,
                        test_ages,
                        test_age_labels
                    )

            output_statistics[subject] = [target_age, output_label]



        except FileNotFoundError as exception_expression:
            if exit_on_error:
                raise(exception_expression)

        except Exception as exception_expression:
            if exit_on_error:
                raise(exception_expression)

    output_statistics_df = pd.DataFrame.from_dict(output_statistics, orient='index', columns=['target_age', 'output_label'])
    output_statistics_df.to_pickle(output_statistics_path)